In [1]:
import numpy as np
import brain_observatory_qc.pipeline_dev.scripts.depth_estimation_module as dem
from tifffile import imread, imsave
from pathlib import Path
import cv2

In [2]:
zstack_dir = Path(r'\\allen\programs\braintv\workgroups\ophysdev\OPhysCore\operator_files\vivian_ha\717487')
zstack_fn = zstack_dir / '20240209_717487_corticalstack_V1_real_00001.tif'

In [3]:
def read_si_stack_metadata(zstack_fn):
    '''Reads metadata from a ScanImage z-stack tiff file.
    
    Args:
        zstack_fn: str or Path, path to the z-stack tiff file
        
    Returns:
        num_slices: int, number of slices in the z-stack
        num_volumes: int, number of volumes in the z-stack
        actuator: str, the actuator used to move the z-stack
        z_values: np.array, the z values of each slice in the z-stack

    TODO: check if actuator and z-values are correct
    '''
    from PIL import Image
    from PIL.TiffTags import TAGS
    with Image.open(zstack_fn) as img:
        meta_dict = {TAGS[key] : img.tag[key] for key in img.tag_v2}
    
    num_slices_ind = np.where(['SI.hStackManager.numSlices = ' in x for x in meta_dict['Software'][0].split('\n')])[0][0]
    num_slices_txt = meta_dict['Software'][0].split('\n')[num_slices_ind]
    num_slices = int(num_slices_txt.split('= ')[1])
    
    num_volumes_ind = np.where(['SI.hStackManager.numVolumes = ' in x for x in meta_dict['Software'][0].split('\n')])[0][0]
    num_volumes_txt = meta_dict['Software'][0].split('\n')[num_volumes_ind]
    num_volumes = int(num_volumes_txt.split('= ')[1])

    actuator_ind = np.where(['SI.hStackManager.stackActuator = ' in x for x in meta_dict['Software'][0].split('\n')])[0][0]
    actuator_txt = meta_dict['Software'][0].split('\n')[actuator_ind]
    actuator = actuator_txt.split('= ')[1].split('\'')[1]

    zs_ind = np.where(['SI.hStackManager.zs = ' in x for x in meta_dict['Software'][0].split('\n')])[0][0]
    zs = meta_dict['Software'][0].split('\n')[zs_ind]
    z_values = np.array([float(z) for z in zs.split('[')[1].split(']')[0].split(' ')])

    return num_slices, num_volumes, actuator, z_values



In [4]:
num_slices, num_volumes, actuator, z_values = read_si_stack_metadata(zstack_fn)

In [5]:
# Takes about 5 min
zstack = imread(zstack_fn)
# it takes > 1 min to get n_frames from PIL (for 42000 frames)
# No num pages from tiff metadata
# opening tiff using cv2 does not make it faster

In [15]:
# Takes about 50 min

assert zstack.shape[0] == num_slices * num_volumes
assert actuator == 'fastZ'
mean_local_zstack_reg = []
for plane_ind in range(num_slices):
    plane_images = zstack[plane_ind::num_slices]
    plane_reg = dem.average_reg_plane(plane_images)
    mean_local_zstack_reg.append(plane_reg)


In [ ]:
zstack_reg = dem.reg_between_planes(np.array(mean_local_zstack_reg))

In [16]:
save_fn = zstack_fn.parent / (zstack_fn.stem + '_reg.tif')
imsave(save_fn, zstack_reg.astype('uint16'))

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_22844\936649946.py:2: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(save_fn, zstack_reg.astype('uint16'))
